# Direct beam iterations for LoKI

## Introduction

This notebook is used to compute the direct beam function for the LoKI detectors.
It uses data recorded during the detector test at the Larmor instrument.

**Description of the procedure:**

The idea behind the direct beam iterations is to determine an efficiency of the detectors as a function of wavelength.
To calculate this, it is possible to compute $I(Q)$ for the full wavelength range, and for individual slices (bands) of the wavelength range.
If the direct beam function used in the $I(Q)$ computation is correct, then $I(Q)$ curves for the full wavelength range and inside the bands should overlap.

In the following notebook, we will:

1. Create two pipelines: one to compute $I(Q)$ for the full wavelength range (`pipeline_full`), and one to compute $I(Q)$ inside a set of wavelength bands (`pipeline_bands`)
1. Create a flat direct beam function, as a function of wavelength, with wavelength bins corresponding to the wavelength bands
1. Calculate inside each band by how much one would have to multiply the final $I(Q)$ so that the curve would overlap with the full range curve
1. Multiply the direct beam values inside each wavelength band by this factor
1. Compare the full-range $I(Q)$ to a theoretical reference and add the corresponding additional scaling to the direct beam function
1. Iterate until the changes to the direct beam function become small

In [ ]:
import numpy as np
import scipp as sc
import sciline
import scippneutron as scn
import plopp as pp
import esssans as sans
from esssans.types import *
from esssans.directbeam import direct_beam

## Define reduction parameters

We define a dictionary containing the reduction parameters, with keys and types given by aliases or types defined in `esssans.types`:

In [ ]:
params = {}

sample_runs = [60339]  # If more runs are added, their events will be merged
sample_filenames = [f'{i}-2022-02-28_2215.nxs' for i in sample_runs]
param_table = sciline.ParamTable(
    SampleRunID, {Filename[SampleRun]: sample_filenames}, index=sample_runs
)

params[Filename[TransmissionRun[SampleRun]]] = '60394-2022-02-28_2215.nxs'
params[Filename[EmptyBeamRun]] = '60392-2022-02-28_2215.nxs'

params[NeXusMonitorName[Incident]] = 'monitor_1'
params[NeXusMonitorName[Transmission]] = 'monitor_2'

# Wavelength binning parameters
wavelength_min = sc.scalar(1.0, unit='angstrom')
wavelength_max = sc.scalar(13.0, unit='angstrom')
n_wavelength_bins = 200
# Wavelength bands parameters
n_wavelength_bands = 50
sampling_width = sc.scalar(0.25, unit='angstrom')

# Wavelength binning and bands
wavelength_bins, wavelength_bands = sans.directbeam.make_wavelength_bins_and_bands(
    wavelength_min=wavelength_min,
    wavelength_max=wavelength_max,
    n_wavelength_bins=n_wavelength_bins,
    n_wavelength_bands=n_wavelength_bands,
    sampling_width=sampling_width,
)

params[WavelengthBins] = wavelength_bins
params[WavelengthBands] = wavelength_bands

params[CorrectForGravity] = True
params[UncertaintyBroadcastMode] = UncertaintyBroadcastMode.upper_bound

params[QBins] = sc.linspace(dim='Q', start=0.01, stop=0.3, num=101, unit='1/angstrom')

# The final result should have dims of Q only
params[FinalDims] = ['Q']

The parameters for the full wavelength range pipeline are almost identical,
except that there is only a single wavelength band that covers the full range.

In [ ]:
params_full = params.copy()
params_full[WavelengthBands] = sc.concat(
    [wavelength_min, wavelength_max], dim='wavelength'
)

## Create pipeline using Sciline

We use all providers available in `esssans` as well as the `loki`-specific providers,
which include I/O and mask setup specific to the [LoKI](https://europeanspallationsource.se/instruments/loki) instrument:

In [ ]:
providers = sans.providers + sans.loki.providers

# Pipeline with wavelength bands
pipeline_bands = sciline.Pipeline(providers, params=params)

# Pipeline for the full wavelength range
pipeline_full = sciline.Pipeline(providers, params=params_full)

pipeline_bands.set_param_table(param_table)
pipeline_full.set_param_table(param_table)

Before we begin computations, we can visualize the pipeline:

In [ ]:
pipeline_full.visualize(IofQ[SampleRun], compact=True, graph_attr={'rankdir': 'LR'})

## Expected intensity at zero Q

The sample used in the experiment has a known $I(Q)$ profile,
and we need it to calibrate the absolute intensity of our $I(Q)$ results
(relative differences between wavelength band and full-range results are not sufficient).

We load this theoretical reference curve, and compute the $I_{0}$ intensity at the lower $Q$ bound of the range covered by the instrument.

In [ ]:
from esssans.data import get_path

ref_filename = 'PolyGauss_I0-50_Rg-60.txt'
data = np.loadtxt(get_path(ref_filename))

qcoord = sc.array(dims=["Q"], values=data[:, 0], unit='1/angstrom')
theory = sc.DataArray(
    data=sc.array(dims=["Q"], values=data[:, 1], unit=''), coords={"Q": qcoord}
)

q_loc = sc.midpoints(params[QBins])[0]
q_loc

ind = np.argmax((qcoord > q_loc).values)
I0 = (theory.data[ind] - theory.data[ind - 1]) / (qcoord[ind] - qcoord[ind - 1]) * (
    q_loc - qcoord[ind - 1]
) + theory.data[ind - 1]
I0

## A single direct beam function for all layers

As a first pass, we compute a single direct beam function for all the detector pixels combined.

We compute the $I(Q)$ using both the `_full` and `_bands` pipelines,
derive a direct beam factor per wavelength band,
and also add absolute scaling using the reference $I_{0}$ value

In [ ]:
results = direct_beam(pipelines=[pipeline_full, pipeline_bands], I0=I0, niter=6)
# Unpack the final result
iofq_full = results[-1]['iofq_full']
iofq_slices = results[-1]['iofq_slices']
direct_beam_function = results[-1]['direct_beam']

We now compare the $I(Q)$ curves in each wavelength band to the one for the full wavelength range (black).

In [ ]:
pp.plot(
    {**sc.collapse(iofq_slices, keep='Q'), **{'full': iofq_full}},
    norm='log',
    color={'full': 'k'},
    legend=False,
)

The overlap is satisfactory, and we can now inspect the direct beam function we have computed:

In [ ]:
direct_beam_function.plot(vmax=1)

Finally, as a sanity check, we compare our final $I(Q)$ for the full wavelength range to the theoretical reference:

In [ ]:
pp.plot({ref_filename: theory, 'data': iofq_full}, norm='log')

## Direct beam function per layer

The LoKI detector tubes are arranged in layers along the beam path,
where the layers closest to the sample will receive most of the scattered neutrons,
while occulting the layers behind them.

A refinement to the above procedure is to compute a direct beam function for each layer of tubes individually.
Because there is only a limited number of events in the run we are using,
we have to reduce the number of wavelength bands we use for the direct beam.
We also use the 4 thick layers of tubes, but in principle,
this could also be done for 28 different layers of tubes if a run with enough events is provided (or many runs are combined together).

So in the following, we use 20 wavelength bands instead of the 50 used above.
The only other difference compared to the computation above is that we now want our final result to preserve the `'layer'` dimension,
so that the dimensions of our result are `['layer', 'Q']`.

In [ ]:
n_wavelength_bands = 20
sampling_width = sc.scalar(0.75, unit='angstrom')

wavelength_bins, wavelength_bands = sans.directbeam.make_wavelength_bins_and_bands(
    wavelength_min=wavelength_min,
    wavelength_max=wavelength_max,
    n_wavelength_bins=n_wavelength_bins,
    n_wavelength_bands=n_wavelength_bands,
    sampling_width=sampling_width,
)

params[WavelengthBins] = wavelength_bins
params[WavelengthBands] = wavelength_bands
# The final result should have dims of (layer, Q)
params[FinalDims] = ['layer', 'Q']

We need to re-build the pipelines from the new parameters:

In [ ]:
params_full = params.copy()
params_full[WavelengthBands] = sc.concat(
    [wavelength_min, wavelength_max], dim='wavelength'
)

providers = sans.providers + sans.loki.providers

# Pipeline with wavelength bands
pipeline_bands = sciline.Pipeline(providers, params=params)

# Pipeline for the full wavelength range
pipeline_full = sciline.Pipeline(providers, params=params_full)

pipeline_bands.set_param_table(param_table)
pipeline_full.set_param_table(param_table)

Now we are able to run the direct-beam iterations on a per-layer basis:

In [ ]:
results = direct_beam(pipelines=[pipeline_full, pipeline_bands], I0=I0, niter=6)
# Unpack the final result
iofq_full = results[-1]['iofq_full']
iofq_slices = results[-1]['iofq_slices']
direct_beam_function = results[-1]['direct_beam']

We can now inspect the wavelength slices for the 4 layers.

In [ ]:
plots = [
    pp.plot(
        {
            **sc.collapse(iofq_slices['layer', i], keep='Q'),
            **{'full': iofq_full['layer', i]},
        },
        norm='log',
        color={'full': 'k'},
        legend=False,
        title=f'Layer {i}',
    )
    for i in range(4)
]

(plots[0] + plots[1]) / (plots[2] + plots[3])

Now the direct beam function inside each layer looks like:

In [ ]:
pp.plot(sc.collapse(direct_beam_function, keep='wavelength'))

And finally, for completeness, we compare the $I(Q)$ to the theoretical reference inside each layer.

In [ ]:
pp.plot({**{ref_filename: theory}, **sc.collapse(iofq_full, keep='Q')}, norm='log')